In [6]:
import json
from collections import Counter
from keras.models import Model
from keras.layers import Embedding,Input,Reshape,Dot
from sklearn.linear_model import LinearRegression
import numpy as np
import random
from sklearn import svm


Load our dataset
(since data is in ndjson file we need json package to load it)

In [7]:
with open('D:/ml_projects/wp_movies/wp_movies_10k.ndjson') as fin:
    movies =[json.loads(l) for l in fin]

In [ ]:
Examine dataset

In [8]:
type(movies)

list

In [17]:
movies[0]

['Deadpool (film)',
 {'image': 'Deadpool poster.jpg',
  'name': 'Deadpool',
  'cinematography': 'Ken Seng',
  'Software Used': 'Adobe Premier Pro',
  'alt': "Official poster shows the titular hero Deadpool standing in front of the viewers, with hugging his hands, and donning his traditional black and red suit and mask, and the film's name, credits and billing below him.",
  'distributor': '20th Century Fox',
  'caption': 'Theatrical release poster',
  'gross': '$783.1 million',
  'country': 'United States',
  'director': 'Tim Miller',
  'runtime': '108 minutes',
  'editing': 'Julian Clarke',
  'language': 'English',
  'music': 'Tom Holkenborg',
  'budget': '$58 million'},
 ['Tim Miller (director)',
  'Simon Kinberg',
  'Ryan Reynolds',
  'Lauren Shuler Donner',
  'Rhett Reese',
  'Paul Wernick',
  'Deadpool',
  'Fabian Nicieza',
  'Rob Liefeld',
  'Morena Baccarin',
  'Ed Skrein',
  'T.J. Miller',
  'Gina Carano',
  'Leslie Uggams',
  'Brianna Hildebrand',
  'Stefan Kapičić',
  'Junkie

In [10]:
movies[0][0]

'Deadpool (film)'

In [11]:
movies[0][1]

{'image': 'Deadpool poster.jpg',
 'name': 'Deadpool',
 'cinematography': 'Ken Seng',
 'Software Used': 'Adobe Premier Pro',
 'alt': "Official poster shows the titular hero Deadpool standing in front of the viewers, with hugging his hands, and donning his traditional black and red suit and mask, and the film's name, credits and billing below him.",
 'distributor': '20th Century Fox',
 'caption': 'Theatrical release poster',
 'gross': '$783.1 million',
 'country': 'United States',
 'director': 'Tim Miller',
 'runtime': '108 minutes',
 'editing': 'Julian Clarke',
 'language': 'English',
 'music': 'Tom Holkenborg',
 'budget': '$58 million'}

In [12]:
movies[0][2]

['Tim Miller (director)',
 'Simon Kinberg',
 'Ryan Reynolds',
 'Lauren Shuler Donner',
 'Rhett Reese',
 'Paul Wernick',
 'Deadpool',
 'Fabian Nicieza',
 'Rob Liefeld',
 'Morena Baccarin',
 'Ed Skrein',
 'T.J. Miller',
 'Gina Carano',
 'Leslie Uggams',
 'Brianna Hildebrand',
 'Stefan Kapičić',
 'Junkie XL',
 'Julian Clarke',
 'Marvel Entertainment',
 'Kinberg Genre',
 'Lauren Shuler Donner',
 'TSG Entertainment',
 '20th Century Fox',
 'Le Grand Rex',
 'Variety (magazine)',
 'Box Office Mojo',
 'superhero film',
 'Tim Miller (director)',
 'Rhett Reese',
 'Paul Wernick',
 'Marvel Comics',
 'Deadpool',
 'X-Men (film series)',
 'Ryan Reynolds',
 'Morena Baccarin',
 'Ed Skrein',
 'T.J. Miller',
 'Gina Carano',
 'Leslie Uggams',
 'Brianna Hildebrand',
 'Stefan Kapičić',
 'antihero',
 'New Line Cinema',
 '20th Century Fox',
 'X-Men Origins: Wolverine',
 'principal photography',
 'Vancouver',
 'IMAX',
 'Digital Light Processing',
 'D-Box Technologies',
 'List of accolades received by Deadpool (

show the most common link counts

In [14]:
link_counts =Counter()

for movie in movies:
    link_counts.update(movie[2])
link_counts.most_common(10)

[('Rotten Tomatoes', 9393),
 ('Category:English-language films', 5882),
 ('Category:American films', 5867),
 ('Variety (magazine)', 5450),
 ('Metacritic', 5112),
 ('Box Office Mojo', 4186),
 ('The New York Times', 3818),
 ('The Hollywood Reporter', 3553),
 ('Roger Ebert', 2707),
 ('Los Angeles Times', 2454)]

# 2. Prepare data for supervised machine learning task
### Map Links to Integers


In [18]:
top_links =[link for link,c in link_counts.items() if c>=3]
print(len(top_links))
top_links[:20]

66913


['Tim Miller (director)',
 'Simon Kinberg',
 'Ryan Reynolds',
 'Lauren Shuler Donner',
 'Rhett Reese',
 'Paul Wernick',
 'Deadpool',
 'Morena Baccarin',
 'Ed Skrein',
 'T.J. Miller',
 'Gina Carano',
 'Leslie Uggams',
 'Brianna Hildebrand',
 'Stefan Kapičić',
 'Junkie XL',
 'Julian Clarke',
 'Marvel Entertainment',
 'Kinberg Genre',
 'TSG Entertainment',
 '20th Century Fox']

In [19]:
#map link to id number
link_to_idx ={link:idx for idx ,link in enumerate(top_links)}

#map movie to id number

movie_to_idx ={movie[0]:idx for idx ,movie in enumerate(movies)}
movie_to_idx

{'Deadpool (film)': 0,
 'The Revenant (2015 film)': 1,
 'Suicide Squad (film)': 2,
 'Spectre (2015 film)': 3,
 'Rebel Without a Cause': 4,
 'Warcraft (film)': 5,
 'The Martian (film)': 6,
 'List of Marvel Cinematic Universe films': 7,
 'X-Men (film series)': 8,
 'The Hateful Eight': 9,
 'The Jungle Book (2016 film)': 10,
 'The Big Short (film)': 11,
 '10 Cloverfield Lane': 12,
 'Spotlight (film)': 13,
 'Room (2015 film)': 14,
 'Creed (film)': 15,
 'DC Universe Animated Original Movies': 16,
 'Star Trek Beyond': 17,
 'Star Wars (film)': 18,
 'Interstellar (film)': 19,
 'Ant-Man (film)': 20,
 'Everest (2015 film)': 21,
 'Jurassic World': 22,
 'Joy (film)': 23,
 'Gods of Egypt (film)': 24,
 'Star Wars sequel trilogy': 25,
 'The Conjuring 2': 26,
 'The Danish Girl (film)': 27,
 'Sicario (2015 film)': 28,
 'Rogue One': 29,
 'Finding Dory': 30,
 'Black Mass (film)': 31,
 'Blade Runner': 32,
 'Harry Potter (film series)': 33,
 'Doctor Strange (film)': 34,
 'Titanic (1997 film)': 35,
 'Furious

Build a traing set

In [21]:
#create a blank array
pairs=[]
for movie in movies:
    pairs.extend((link_to_idx[link],movie_to_idx[movie[0]])for link in movie[2] if link in link_to_idx)
pairs_set=set(pairs)
len(pairs),len(top_links),len(movie_to_idx)

(949544, 66913, 10000)

In [22]:
pairs_set

{(23724, 7080),
 (4398, 5536),
 (1801, 3924),
 (5414, 8627),
 (88, 2598),
 (18906, 9593),
 (22, 1978),
 (170, 4882),
 (12188, 6043),
 (63516, 6293),
 (12712, 4508),
 (20331, 811),
 (28956, 1739),
 (63200, 6996),
 (55038, 7290),
 (193, 2799),
 (11621, 9379),
 (1077, 833),
 (975, 556),
 (19772, 2510),
 (1232, 1121),
 (35611, 1907),
 (53730, 4929),
 (2617, 2389),
 (196, 37),
 (30165, 2192),
 (3248, 5639),
 (30003, 3362),
 (65271, 8942),
 (541, 449),
 (8755, 345),
 (28053, 595),
 (13385, 1194),
 (4295, 3879),
 (18809, 2813),
 (47778, 5793),
 (12694, 2774),
 (736, 166),
 (1146, 8143),
 (52844, 3008),
 (10658, 3646),
 (8612, 417),
 (2039, 5916),
 (22641, 1588),
 (855, 4180),
 (18933, 785),
 (24082, 3889),
 (34378, 6671),
 (1076, 2836),
 (31301, 9118),
 (15518, 204),
 (20611, 2275),
 (36551, 8894),
 (48225, 6081),
 (23010, 6531),
 (1933, 952),
 (29916, 683),
 (2866, 8704),
 (3045, 4505),
 (37547, 3382),
 (911, 1429),
 (7695, 6730),
 (749, 347),
 (3046, 943),
 (22, 18),
 (44011, 6988),
 (964, 

Build the Entity embedding Neural Network & Train Model

In [25]:
def movie_embedding_model(embedding_size=50):
    link=Input(name='link',shape=(1,))
    movie =Input(name='movie',shape=(1,))
    link_embedding=Embedding(name='link_embedding', 
                             input_dim=len(top_links),
                             output_dim=embedding_size)(link)
    movie_embedding=Embedding(name='movie_embedding', 
                             input_dim=len(movie_to_idx),
                             output_dim=embedding_size)(movie)
    dot =Dot(name='dot_product',normalize=True,axes=2)([link_embedding,movie_embedding])
    merged=Reshape((1,))(dot)
    model =Model(inputs=[link,movie],outputs=[merged])
    model.compile(optimizer='nadam',loss='mse')
    
    return model

model=movie_embedding_model()
model.summary()
    

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 link (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 movie (InputLayer)          [(None, 1)]                  0         []                            
                                                                                                  
 link_embedding (Embedding)  (None, 1, 50)                3345650   ['link[0][0]']                
                                                                                                  
 movie_embedding (Embedding  (None, 1, 50)                500000    ['movie[0][0]']               
 )                                                                                          

In [31]:
random.seed(5)

def batchifier(pairs, positive_samples=50, negative_ratio=10):
    batch_size = positive_samples * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    while True:
        for idx, (link_id, movie_id) in enumerate(random.sample(pairs, positive_samples)):
            batch[idx, :] = (link_id, movie_id, 1)
        idx = positive_samples
        while idx < batch_size:
            movie_id = random.randrange(len(movie_to_idx))
            
            if not (link_id, movie_id) in pairs_set:
                batch[idx, :] = (link_id, movie_id, -1)
                idx += 1
        np.random.shuffle(batch)
        yield {'link': batch[:, 0], 'movie': batch[:, 1]}, batch[:, 2]

next(batchifier(pairs, positive_samples=3, negative_ratio=2))

({'link': array([ 1313., 31254., 22418.,  1313.,  1313.,  1313.,  1313.,  1313.,
          1313.]),
  'movie': array([8684., 5530., 1529., 5874., 7236.,  849.,  475., 7628., 4080.])},
 array([-1.,  1.,  1., -1.,  1., -1., -1., -1., -1.]))

Train model

In [32]:
positive_samples_per_batch =512

model.fit_generator(
    batchifier(pairs,positive_samples=positive_samples_per_batch,negative_ratio=10),
    epochs=5,
    steps_per_epoch=len(pairs)//positive_samples_per_batch,
    verbose=1
)

C:\Users\pavit\AppData\Local\Temp\ipykernel_18156\1860476369.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
1854/1854 [==============================] - 200s 107ms/step - loss: 0.9970
Epoch 2/5
1854/1854 [==============================] - 198s 107ms/step - loss: 0.9731
Epoch 3/5
1854/1854 [==============================] - 196s 106ms/step - loss: 0.9584
Epoch 4/5
1854/1854 [==============================] - 219s 118ms/step - loss: 0.9550
Epoch 5/5
1854/1854 [==============================] - 203s 110ms/step - loss: 0.9550


Save model

In [33]:
model.save('movie_embedding.h5')

C:\Users\pavit\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 Extract embeddings and Finding Similar Movies

In [36]:
# Get the Movie embeddings layers 
movie = model.get_layer('movie_embedding')

# Get the weights 
movie_weights = movie.get_weights()[0]

# Get the matrix normal of movie_weights 
movie_lengths = np.linalg.norm(movie_weights, axis=1)

# Get the normalized movie embeddigns 
normalized_movies = (movie_weights.T / movie_lengths).T

def similar_movies(movie,normalized_movies):
    #this represents the cosine similarity of the two vectors
    dists = np.dot(normalized_movies,normalized_movies[movie_to_idx[movie]])
    
    #SORT DOT PRODUCT RESULTS TO GET THE MOST SIMILAR MOVIES
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c,movies[c][0],dists[c])

similar_movies("Interstellar (film)", normalized_movies)

19 Interstellar (film) 1.0000001
3349 Star Wars: The Force Awakens 0.76516974
42 The Avengers (2012 film) 0.7531577
85 Inception 0.74115396
78 The Dark Knight Rises 0.7255415
39 Guardians of the Galaxy (film) 0.72525066
22 Jurassic World 0.70621675
18 Star Wars (film) 0.6900856
1364 Captain America: Civil War 0.6854873
7 List of Marvel Cinematic Universe films 0.6835373


wikilink embeddings

In [37]:
link = model.get_layer('link_embedding')
link_weights = link.get_weights()[0]
link_lengths = np.linalg.norm(link_weights, axis=1)
normalized_links = (link_weights.T / link_lengths).T

def similar_links(link):
    dists = np.dot(normalized_links, normalized_links[link_to_idx[link]])
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c, top_links[c], dists[c])

similar_links('George Lucas')

127 George Lucas 1.0000001
19260 Category:Indiana Jones films 0.8623913
2778 Lucasfilm 0.8504857
3176 Star Wars (film) 0.84416467
3086 Category:Lucasfilm films 0.8354355
2707 Star Wars 0.83358467
19209 Marcus Brody 0.8333764
5693 James Kahn 0.8307277
2704 Lucasfilm Ltd. 0.8246367
2824 Willard Huyck 0.818249
